### 1. 题目描述

The **Employee** table holds all employees. Every employee has an Id, and there is also a column for the department Id.


| Id | Name  | Salary | DepartmentId |
|----|-------|--------|--------------|
| 1  | Joe   | 70000  | 1         |
| 2  | Henry | 80000  | 2         |
| 3  | Sam   | 60000  | 2         |
| 4  | Max   | 90000  | 1         |
| 5  | Janet | 69000  | 1         |
| 6  | Randy | 85000  | 1         |


The **Department** table holds all departments of the company.


| Id | Name     |
|----|----------|
| 1  | IT       |
| 2  | Sales    |


Write a SQL query to find employees who earn the top three salaries in each of the department. For the above tables, your SQL query should return the following rows.


| Department | Employee | Salary |
|------------|----------|--------|
| IT         | Max      | 90000  |
| IT         | Randy    | 85000  |
| IT         | Joe      | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |


### 2. 创建数据

```sql
create database if not exists test;
use test;

create table `Employee` (
    `Id` bigint,
    `Name` varchar(10) ,
    `Salary` int,
    `DepartmentId` bigint
) engine = InnoDB default charset = utf8;

create table `Department` (
    `Id` bigint, 
    `Name` varchar(10)
)engine = InnoDB default charset = utf8;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (1,	'Joe',	70000,	1)
;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (2,	'Henry',	80000,	2)
;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (3,	'Sam',	60000,	2)
;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (4,	'Max',	90000,	1)
;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (5,	'Janet',	69000,	1)
;

insert into `Employee` (`Id`, `Name`, `Salary`, `DepartmentId`)
values (6,	'Randy',	85000,	1)
;

insert into `Department` (`Id`, `Name`)
values (1,	'IT')
;

insert into `Department` (`Id`, `Name`)
values (2,	'Sales')
;
```

### 3. 解题方法

#### 3.1 相关子查询方法

在相关子查询中，找到和外部查询得到的部门一样，并且比外部查询得到的薪水高的个数。

```sql
select  t2.Name as Department, t1.Name as Employee, t1.Salary as Salary
from   Employee t1
      inner join
      Department t2 on t2.Id = t1.DepartmentId
where  ( select count(distinct Salary)
      from  Employee
      where Salary > t1.Salary
          and DepartmentId = t2.Id
      ) < 3
order by t2.Name, t1.Salary desc
;
```

```sql
select  t2.Name as Department, t1.Name as Employee, t1.Salary as Salary
from   Employee t1
      inner join
      Department t2 on t2.Id = t1.DepartmentId
where  ( select count(distinct Salary)
      from  Employee
      where Salary > t1.Salary
          and DepartmentId = t2.Id
      ) in (0,1,2)
order by t2.Name, t1.Salary desc
;
```

#### 3.2 设置变量的方法

```sql
set @salary_rank=0;
set @current_id=NULL;
set @previous_salary=-1;

select Department, Employee, Salary
from  ( select Department, Employee, Salary,
           /*RANK() OVER (PARTITION BY E.Department ORDER BY E.Salary DESC) SalaryRank*/
           @salary_rank := IF(@current_id = Department,IF(@previous_salary = Salary, @salary_rank, @salary_rank + 1), 1) AS SalaryRank,
           @current_id := Department,
           @previous_salary := Salary
      from ( select D.Name Department, E.Name Employee, E.Salary Salary
           from  Employee E
               inner join Department D on D.Id = E.DepartmentId
           group by D.Name, E.Salary, E.Name
           order by D.Name asc, E.Salary desc
          ) E
     ) R
where SalaryRank <= 3
```

```sql
drop database test;
```